In [1]:
import requests
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from CertificationAuthority import CertificationAuthority
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
import util


In [2]:
#ca_root use port 8000
#ca1 use port 8001
#ca2 use port 8002
#ca1_1 use port 8003
#ca1_2 use port 8004

# cleint 1 cert sign by root
# 1. generate RSA key
rsa_key=util.generate_ras_key()

# 2. create certificate sign request and sign with client private key
csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"California"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"client1.com"),
])).add_extension(
    x509.SubjectAlternativeName([
        # Describe what sites we want this certificate for.
        x509.DNSName(u"mysite.com"),
        x509.DNSName(u"www.mysite.com"),
        x509.DNSName(u"subdomain.mysite.com"),
    ]),
    critical=False,
# Sign the CSR with our private key. is applicant's private key
).sign(rsa_key, hashes.SHA256())

# 3. send to certificate authority,  the data param name corresond fastapi parameter
response = requests.post(" http://127.0.0.1:8003/issue_cert",files={'csr_file':csr.public_bytes(serialization.Encoding.PEM)})
cert = x509.load_pem_x509_certificate(response.content)
print("applicant: "+cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(cert.not_valid_before)
print(cert.not_valid_after)

applicant: client1.com
issuer: ca1_1
2021-11-24 05:27:03
2021-12-04 05:27:03


In [10]:
# cleint 2 cert sign by ca1
# 1. generate RSA key
rsa_key=util.generate_ras_key()

# 2. create certificate sign request and sign with client private key
csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"California"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"client2.com"),
])).add_extension(
    x509.SubjectAlternativeName([
        # Describe what sites we want this certificate for.
        x509.DNSName(u"mysite.com"),
        x509.DNSName(u"www.mysite.com"),
        x509.DNSName(u"subdomain.mysite.com"),
    ]),
    critical=False,
# Sign the CSR with our private key. is applicant's private key
).sign(rsa_key, hashes.SHA256())

# 3. send to certificate authority,  the data param name corresond fastapi parameter
## notice here port is 8001
response = requests.post(" http://127.0.0.1:8001/issue_cert",files={'csr_file':csr.public_bytes(serialization.Encoding.PEM)})
client1_cert = x509.load_pem_x509_certificate(response.content)
print("applicant: "+client1_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+client1_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client1_cert.not_valid_before)
print(client1_cert.not_valid_after)

applicant: client2.com
issuer: ca1
2021-11-23 14:41:17
2021-12-03 14:41:17


In [11]:
response = requests.post(" http://127.0.0.1:8000/get_CA_public_key")
root_public_key=serialization.load_pem_public_key(response.content)
response = requests.post(" http://127.0.0.1:8000/get_CA_cert")
ca_root_cert = x509.load_pem_x509_certificate(response.content)

response = requests.post(" http://127.0.0.1:8001/get_CA_public_key")
ca1_public_key=serialization.load_pem_public_key(response.content)
response = requests.post(" http://127.0.0.1:8001/get_CA_cert")
ca1_cert = x509.load_pem_x509_certificate(response.content)


# verify cert chain
print("\n =======verify cert chain ===========")
print('client 1 cert verify with root public key')
print("validate: "+ str(util.verify_cert_signature(client1_cert,root_public_key)))
print('client 1 cert verify with CA1 public key')
print("validate: "+ str(util.verify_cert_signature(client1_cert,ca1_public_key)))
print('CA1 cert verify with root public key')
print("validate: "+ str(util.verify_cert_signature(ca1_cert,root_public_key)))
print('revoke CA1 cert in root')




 =======verify cert chain ===========
client 1 cert verify with root public key
validate: False
client 1 cert verify with CA1 public key
validate: True
CA1 cert verify with root public key
validate: True
revoke CA1 cert in root


In [12]:
response = requests.post(" http://127.0.0.1:8000/revoke_cert_status",files={'crt_file':ca1_cert.public_bytes(serialization.Encoding.PEM)})
print(response.content)
print()
response = requests.post(" http://127.0.0.1:8000/revoke_cert",files={'crt_file':ca1_cert.public_bytes(serialization.Encoding.PEM)})
print(response.status_code)
print(response.content)
print('# revoke again #')
print(response.status_code)
response = requests.post(" http://127.0.0.1:8000/revoke_cert",files={'crt_file':ca1_cert.public_bytes(serialization.Encoding.PEM)})
print(response.content)
print()

b'{"msg":"Not revoke"}'

200
b'{"msg":"success"}'
# revoke again #
200
b'{"msg":"Already revoke"}'

